Script L4B MTZ para Emissão NFTS SP.

In [3]:
import pandas as pd

df = pd.read_excel('G:\\Meu Drive\\Projetos\\NFTS_SP\\SAP.xlsx', dtype = str)
cod_sp = pd.read_excel('G:\\Meu Drive\\Projetos\\NFTS_SP\\Arquivos\\CODSP.xlsx', dtype = str)
c_Prestador = pd.read_excel('G:\\Meu Drive\\Projetos\\NFTS_SP\Arquivos\\c_Prestadores.xlsx', dtype = str)

# Datas em formato americano.
dtint = input('Data inicio: ')
dtfim = input('Data fim: ')

#Alterar tipo de dado da coluna.
df = df.astype({
    'NºID fiscal': str,
    'Número de nota fiscal eletrônica': str,
    'ISS': float,
    'Valor_Serv': float
})

#Criação dos filtros, seleção das colunas e renomear as mesmas.
df = df.filter(items = ['Descrição Central', 'Cidade', 'NºID fiscal', 'Data documento', 
                        'Número de nota fiscal eletrônica', 'Valor_Serv', '%ISS','ISS', 'LC 116 código de tipo de serviço', 
                        'Sub item_SP','CFOP', 'Texto breve', 'Documento MIRO'])

df.rename(columns= {'NºID fiscal' : 'CNPJ', 'Número de nota fiscal eletrônica' : 'Nota', 
                    'Texto breve' : 'Descrição do Serviço', 'LC 116 código de tipo de serviço' : 'LC 116/2003' }, inplace=True)

df[['Data documento', 'HORA']] = df['Data documento'].str.split(' ', expand= True)

#Seleção Filial e filtros.
df = df.loc[df['Descrição Central'] == 'MTZ - 0001 SAO PAULO']
df = df.loc[df['Cidade'] != 'SAO PAULO']
df = df[df['CFOP'].isin(['1933/AA' , '2933/AA'])]
df['%ISS'] = df['%ISS'].str.rstrip('%').astype(float)

#Criação das colunas personalizadas, parâmetros conforme manual.
df.insert(3, 'T_Registro', '4')
df.insert(4, 'T_Documento', '02')
df.insert(5, 'Serie', '    E')
df.insert(6, 'SNFTS', 'N')
df.insert(7, 'Tributação', 'T')
df.insert(11, 'vDeducao', '000000000000000')
df.insert(4, 'Ind_CNPJ', '2')
df.insert(8, 'Regime_T', '0')
df.insert(11, 'Tipo_End', 'END')
df.insert(9, 'Complemento', ' ')
df.insert(12, 'CCM', '00000000')
df.insert(13, 'T_NFTS', '1')
df.insert(14, 'C_INSS', '0')
df.insert(15, 'N_OBRA', '0')
df.insert(15, 'C_RESERVADO', ' ')
df.insert(16, 'Data_PGTO', ' ')

#Criação das colunas conforme regra do valor.
df.loc[df['ISS'] > 0, 'ISS Retido'] = '1'
df.loc[df['ISS'] == 0, 'ISS Retido'] = '2'
df['Valor_Serv'] = df['Valor_Serv'].apply(lambda x: format(round(x, 2), '.2f'))
df['%ISS'] = (df['%ISS'] * 100).round().astype(int)

#União das planilha base trazendo os subitens COD.
df = pd.merge(df, cod_sp, on='LC 116/2003', how='inner')
df = pd.merge(df, c_Prestador, on='CNPJ', how='inner')

df.drop(['HORA', 'DESCRIÇÃO'], axis= 1, inplace= True)
df.drop_duplicates(['Documento MIRO'], inplace=True)

df['Data documento'] = df['Data documento'].str.replace('-', '')
df['LC 116/2003'] = df['LC 116/2003'].str.replace('.', '')
df['CEP'] = df['CEP'].str.replace('.', '')
df['CEP'] = df['CEP'].str.replace('-', '')
df['Valor_Serv'] = df['Valor_Serv'].str.replace('.', '')

#Parametrização das colunas utilizando 'PAD'
df['Nota'] = df['Nota'].str.pad(width=12, side='left', fillchar='0')
df['CNPJ'] = df['CNPJ'].str.pad(width=14, side='left', fillchar='0')
df['Valor_Serv'] = df['Valor_Serv'].str.pad(width=15, side='left', fillchar='0')
df['CÓDIGO'] = df['CÓDIGO'].str.pad(width=5, side='left', fillchar='0')
df['%ISS'] = df['%ISS'].apply(lambda x: str(x).zfill(4))
df['RAZÃO SOCIAL'] = df['RAZÃO SOCIAL'].str.pad(width=75, side='right', fillchar=' ')
df['LOGRADOURO'] = df['LOGRADOURO'].str.pad(width=50, side='right', fillchar=' ')
df['NÚMERO'] = df['NÚMERO'].str.pad(width=10, side='right', fillchar=' ')
df['Complemento'] = df['Complemento'].str.pad(width=30, side='right', fillchar=' ')
df['BAIRRO'] = df['BAIRRO'].str.pad(width=30, side='right', fillchar=' ')
df['CIDADE'] = df['CIDADE'].str.pad(width=50, side='right', fillchar=' ')
df['EMAIL'] = df['EMAIL'].astype(str).apply(lambda x: x.ljust(75))
df['C_INSS'] = df['C_INSS'].str.pad(width=12, side='right', fillchar='0')
df['N_OBRA'] = df['N_OBRA'].str.pad(width=12, side='right', fillchar='0')
df['C_RESERVADO'] = df['C_RESERVADO'].str.pad(width=200, side='right', fillchar=' ')
df['Descrição do Serviço'] = df['Descrição do Serviço'].str.pad(width=200, side='right', fillchar=' ')
df['Data_PGTO'] = df['Data_PGTO'].str.pad(width=8, side='right', fillchar=' ')
df['UF'] = df['UF'].str.pad(width=2, side='right', fillchar=' ')
df['CEP'] = df['CEP'].str.pad(width=8, side='right', fillchar='0')

dfNFTS = pd.DataFrame()

# LL4B IM Municipal - 100254271860

dfNFTS[f'100254271860{dtint}{dtfim}'] = pd.concat([df['T_Registro'], df['T_Documento'], df['Serie'], df['Nota'], df['Data documento'], df['SNFTS'],
df['Tributação'], df['Valor_Serv'], df['vDeducao'], df['CÓDIGO'], df['LC 116/2003'], df['%ISS'],
df['ISS Retido'], df['Ind_CNPJ'], df['CNPJ'], df['CCM'], df['RAZÃO SOCIAL'], df['Tipo_End'], df['LOGRADOURO'],
df['NÚMERO'], df['Complemento'], df['BAIRRO'], df['CIDADE'], df['UF'], df['CEP'], df['EMAIL'],
df['T_NFTS'], df['Regime_T'], df['Data_PGTO'], df['C_INSS'], df['N_OBRA'], df['C_RESERVADO'], df['Descrição do Serviço']], 
axis=1, ignore_index=True).apply(lambda x: ''.join(str(val) for val in x), axis=1)

dfNFTS[f'100254271860{dtint}{dtfim}'] = dfNFTS[f'100254271860{dtint}{dtfim}'].str.pad(width=900, side='right', fillchar=' ')

dfNFTS.to_csv('LL4BNFTS_SP.txt', index=False)

Script LTEC MTZ para Emissão NFTS SP.

In [ ]:
import pandas as pd

df = pd.read_excel('G:\\Meu Drive\\Projetos\\NFTS_SP\\SAP.xlsx', dtype = str)
cod_sp = pd.read_excel('G:\\Meu Drive\\Projetos\\NFTS_SP\\Arquivos\\CODSP.xlsx', dtype = str)
c_Prestador = pd.read_excel('G:\\Meu Drive\\Projetos\\NFTS_SP\Arquivos\\c_Prestadores.xlsx', dtype = str)

# Datas em formato americano.
dtint = input('Data inicio: ')
dtfim = input('Data fim: ')

#Alterar tipo de dado da coluna.
df = df.astype({
    'NºID fiscal': str,
    'Número de nota fiscal eletrônica': str,
    'ISS': float,
    'Valor_Serv': float
})

#Criação dos filtros, seleção das colunas e renomear as mesmas.
df = df.filter(items = ['Descrição Central', 'Cidade', 'NºID fiscal', 'Data documento', 
                        'Número de nota fiscal eletrônica', 'Valor_Serv', '%ISS','ISS', 'LC 116 código de tipo de serviço', 
                        'Sub item_SP','CFOP', 'Texto breve', 'Documento MIRO'])

df.rename(columns= {'NºID fiscal' : 'CNPJ', 'Número de nota fiscal eletrônica' : 'Nota', 
                    'Texto breve' : 'Descrição do Serviço', 'LC 116 código de tipo de serviço' : 'LC 116/2003' }, inplace=True)

df[['Data documento', 'HORA']] = df['Data documento'].str.split(' ', expand= True)

#Seleção Filial e filtros.
df = df.loc[df['Descrição Central'] == 'MTZ - 0001 LOGGI TEC LTDA']
df = df.loc[df['Cidade'] != 'SAO PAULO']
df = df[df['CFOP'].isin(['1933/AA' , '2933/AA'])]
df['%ISS'] = df['%ISS'].str.rstrip('%').astype(float)

#Criação das colunas personalizadas, parâmetros conforme manual.
df.insert(3, 'T_Registro', '4')
df.insert(4, 'T_Documento', '02')
df.insert(5, 'Serie', '    E')
df.insert(6, 'SNFTS', 'N')
df.insert(7, 'Tributação', 'T')
df.insert(11, 'vDeducao', '000000000000000')
df.insert(4, 'Ind_CNPJ', '2')
df.insert(8, 'Regime_T', '0')
df.insert(11, 'Tipo_End', 'END')
df.insert(9, 'Complemento', ' ')
df.insert(12, 'CCM', '00000000')
df.insert(13, 'T_NFTS', '1')
df.insert(14, 'C_INSS', '0')
df.insert(15, 'N_OBRA', '0')
df.insert(15, 'C_RESERVADO', ' ')
df.insert(16, 'Data_PGTO', ' ')

#Criação das colunas conforme regra do valor.
df.loc[df['ISS'] > 0, 'ISS Retido'] = '1'
df.loc[df['ISS'] == 0, 'ISS Retido'] = '2'
df['Valor_Serv'] = df['Valor_Serv'].apply(lambda x: format(round(x, 2), '.2f'))
df['%ISS'] = (df['%ISS'] * 100).round().astype(int)

#União das planilha base trazendo os subitens COD.
df = pd.merge(df, cod_sp, on='LC 116/2003', how='inner')
df = pd.merge(df, c_Prestador, on='CNPJ', how='inner')

df.drop(['HORA', 'DESCRIÇÃO'], axis= 1, inplace= True)
df.drop_duplicates(['Documento MIRO'], inplace=True)

df['Data documento'] = df['Data documento'].str.replace('-', '')
df['LC 116/2003'] = df['LC 116/2003'].str.replace('.', '')
df['CEP'] = df['CEP'].str.replace('.', '')
df['CEP'] = df['CEP'].str.replace('-', '')
df['Valor_Serv'] = df['Valor_Serv'].str.replace('.', '')

#Parametrização das colunas utilizando 'PAD'
df['Nota'] = df['Nota'].str.pad(width=12, side='left', fillchar='0')
df['CNPJ'] = df['CNPJ'].str.pad(width=14, side='left', fillchar='0')
df['Valor_Serv'] = df['Valor_Serv'].str.pad(width=15, side='left', fillchar='0')
df['CÓDIGO'] = df['CÓDIGO'].str.pad(width=5, side='left', fillchar='0')
df['%ISS'] = df['%ISS'].apply(lambda x: str(x).zfill(4))
df['RAZÃO SOCIAL'] = df['RAZÃO SOCIAL'].str.pad(width=75, side='right', fillchar=' ')
df['LOGRADOURO'] = df['LOGRADOURO'].str.pad(width=50, side='right', fillchar=' ')
df['NÚMERO'] = df['NÚMERO'].str.pad(width=10, side='right', fillchar=' ')
df['Complemento'] = df['Complemento'].str.pad(width=30, side='right', fillchar=' ')
df['BAIRRO'] = df['BAIRRO'].str.pad(width=30, side='right', fillchar=' ')
df['CIDADE'] = df['CIDADE'].str.pad(width=50, side='right', fillchar=' ')
df['EMAIL'] = df['EMAIL'].astype(str).apply(lambda x: x.ljust(75))
df['C_INSS'] = df['C_INSS'].str.pad(width=12, side='right', fillchar='0')
df['N_OBRA'] = df['N_OBRA'].str.pad(width=12, side='right', fillchar='0')
df['C_RESERVADO'] = df['C_RESERVADO'].str.pad(width=200, side='right', fillchar=' ')
df['Descrição do Serviço'] = df['Descrição do Serviço'].str.pad(width=200, side='right', fillchar=' ')
df['Data_PGTO'] = df['Data_PGTO'].str.pad(width=8, side='right', fillchar=' ')
df['UF'] = df['UF'].str.pad(width=2, side='right', fillchar=' ')
df['CEP'] = df['CEP'].str.pad(width=8, side='right', fillchar='0')

dfNFTS = pd.DataFrame()

# LTEC IM Municipal - 100254271860

dfNFTS[f'100259871091{dtint}{dtfim}'] = pd.concat([df['T_Registro'], df['T_Documento'], df['Serie'], df['Nota'], df['Data documento'], df['SNFTS'],
                                                    df['Tributação'], df['Valor_Serv'], df['vDeducao'], df['CÓDIGO'], df['LC 116/2003'], df['%ISS'],
                                                    df['ISS Retido'], df['Ind_CNPJ'], df['CNPJ'], df['CCM'], df['RAZÃO SOCIAL'], df['Tipo_End'], df['LOGRADOURO'],
                                                    df['NÚMERO'], df['Complemento'], df['BAIRRO'], df['CIDADE'], df['UF'], df['CEP'], df['EMAIL'],
                                                    df['T_NFTS'], df['Regime_T'], df['Data_PGTO'], df['C_INSS'], df['N_OBRA'], df['C_RESERVADO'], df['Descrição do Serviço']], 
                                                    axis=1, ignore_index=True).apply(lambda x: ''.join(str(val) for val in x), axis=1)

dfNFTS[f'100259871091{dtint}{dtfim}'] = dfNFTS[f'100259871091{dtint}{dtfim}'].str.pad(width=900, side='right', fillchar=' ')

dfNFTS.to_csv('LTECNFTS_SP.txt', index=False)

Script L4B 0006 e 0100 NFTS Cajamar.

In [13]:
df = pd.read_excel('G:\\Meu Drive\\Projetos\\NFTS_SP\\SAP.xlsx', dtype = str)

#Alterar tipo de dado da coluna.
df = df.astype({
    'NºID fiscal': str,
    'ISS': float,
    'Valor_Serv': float
})

#Criação dos filtros, seleção das colunas e renomear as mesmas.
df = df.filter(items = ['Descrição Central', 'Cidade', 'NºID fiscal', 'Data documento', 
                        'Número de nota fiscal eletrônica', 'Valor_Serv', '%ISS','ISS', 'LC 116 código de tipo de serviço', 
                        'CFOP', 'Texto breve', 'Documento MIRO'])

df.rename(columns= {'NºID fiscal' : 'CNPJ', 'Número de nota fiscal eletrônica' : 'Nota',
                    'Texto breve' : 'Descrição do Serviço', 'LC 116 código de tipo de serviço' : 'LC 116/2003' }, inplace=True)

df[['Data documento', 'HORA']] = df['Data documento'].str.split(' ', expand= True)

#Seleção Filial e filtros.
df = df.loc[df['Descrição Central'] == 'XD - 0006 CAJAMAR']
df = df.loc[df['Cidade'] != 'CAJAMAR']
df = df[df['CFOP'].isin(['1933/AA' , '2933/AA'])]

#Criação das colunas personalizadas, parâmetros conforme manual.
df.insert(13, 'T_NFTS', 'NFSE ')
df.insert(13, 'Serie', 'E ')
df.insert(13, 'Municipio_T', 'Cajamar')
df.insert(13, 'CNPJ_Filial', '24217653000608')
df.insert(13, 'D_Lançamento', '20230227')
df.loc[df['ISS'] > 0, 'ISS Retido'] = 'T'
df.loc[df['ISS'] == 0, 'ISS Retido'] = 'R'

#Criação das colunas conforme regra do valor.
df['Valor_Serv'] = df['Valor_Serv'].apply(lambda x: format(round(x, 2), '.2f'))

df.drop(['HORA'], axis= 1, inplace= True)
df.drop_duplicates(['Documento MIRO'], inplace=True)

df['Data documento'] = df['Data documento'].str.replace('-', '')
df['LC 116/2003'] = df['LC 116/2003'].str.replace('.', '')
df['Valor_Serv'] = df['Valor_Serv'].str.replace('.', '')

#Parametrização das colunas utilizando 'PAD'
df['Nota'] = df['Nota'].str.pad(width=6, side='left', fillchar='0')
df['Valor_Serv'] = df['Valor_Serv'].str.pad(width=14, side='left', fillchar='0')
df['CNPJ'] = df['CNPJ'].str.pad(width=14, side='left', fillchar='0')
df['Municipio_T'] = df['Municipio_T'].str.pad(width=50, side='right', fillchar=' ')

dfNFTS = pd.Series(df['CNPJ_Filial'] + df['T_NFTS'] + df['Serie'] + df['Nota'] + df['LC 116/2003'] + df['ISS Retido'] +
                    df['Data documento'] + df['Valor_Serv'] + df['Valor_Serv'] + df['CNPJ'] + df['Municipio_T'] + df['D_Lançamento'])

dfNFTS.to_csv('AG06NFTS.txt', index=False)

In [14]:
df = pd.read_excel('G:\\Meu Drive\\Projetos\\NFTS_SP\\SAP.xlsx', dtype = str)

#Alterar tipo de dado da coluna.
df = df.astype({
    'NºID fiscal': str,
    'ISS': float,
    'Valor_Serv': float
})

#Criação dos filtros, seleção das colunas e renomear as mesmas.
df = df.filter(items = ['Descrição Central', 'Cidade', 'NºID fiscal', 'Data documento', 
                        'Número de nota fiscal eletrônica', 'Valor_Serv', '%ISS','ISS', 'LC 116 código de tipo de serviço', 
                        'CFOP', 'Texto breve', 'Documento MIRO'])

df.rename(columns= {'NºID fiscal' : 'CNPJ', 'Número de nota fiscal eletrônica' : 'Nota',
                    'Texto breve' : 'Descrição do Serviço', 'LC 116 código de tipo de serviço' : 'LC 116/2003' }, inplace=True)

df[['Data documento', 'HORA']] = df['Data documento'].str.split(' ', expand= True)

#Seleção Filial e filtros.
df = df.loc[df['Descrição Central'] == 'XD - 0100 CAJAMAR II']
df = df.loc[df['Cidade'] != 'CAJAMAR']
df = df[df['CFOP'].isin(['1933/AA' , '2933/AA'])]

#Criação das colunas personalizadas, parâmetros conforme manual.
df.insert(13, 'T_NFTS', 'NFSE ')
df.insert(13, 'Serie', 'E ')
df.insert(13, 'Municipio_T', 'Cajamar')
df.insert(13, 'CNPJ_Filial', '24217653010077')
df.insert(13, 'D_Lançamento', '20230227')
df.loc[df['ISS'] > 0, 'ISS Retido'] = 'T'
df.loc[df['ISS'] == 0, 'ISS Retido'] = 'R'

#Criação das colunas conforme regra do valor.
df['Valor_Serv'] = df['Valor_Serv'].apply(lambda x: format(round(x, 2), '.2f'))

df.drop(['HORA'], axis= 1, inplace= True)
df.drop_duplicates(['Documento MIRO'], inplace=True)

df['Data documento'] = df['Data documento'].str.replace('-', '')
df['LC 116/2003'] = df['LC 116/2003'].str.replace('.', '')
df['Valor_Serv'] = df['Valor_Serv'].str.replace('.', '')

#Parametrização das colunas utilizando 'PAD'
df['Nota'] = df['Nota'].str.pad(width=6, side='left', fillchar='0')
df['Valor_Serv'] = df['Valor_Serv'].str.pad(width=14, side='left', fillchar='0')
df['CNPJ'] = df['CNPJ'].str.pad(width=14, side='left', fillchar='0')
df['Municipio_T'] = df['Municipio_T'].str.pad(width=50, side='right', fillchar=' ')

dfNFTS = pd.Series(df['CNPJ_Filial'] + df['T_NFTS'] + df['Serie'] + df['Nota'] + df['LC 116/2003'] + df['ISS Retido'] +
                    df['Data documento'] + df['Valor_Serv'] + df['Valor_Serv'] + df['CNPJ'] + df['Municipio_T'] + df['D_Lançamento'])

dfNFTS.to_csv('AG100NFTS.txt', index=False)

NFTS Importações

In [33]:
from __future__ import print_function
import os.path
import pandas as pd
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
# Ao modificar esses escopos, exclua o arquivo token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
'''Mostra o uso básico da Sheets API.
Imprime valores de uma planilha de amostra.'''
creds = None
'''O arquivo token.json armazena os tokens de acesso e atualização do usuário e é criado 
automaticamente quando o fluxo de autorização é concluído pela primeira vez.'''
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# Se não houver credenciais (válidas) disponíveis, deixe o usuário efetuar login.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
# Salva as credenciais para a próxima execução
    with open('token.json', 'w') as token:
        token.write(creds.to_json())
try:
    service = build('sheets', 'v4', credentials=creds)
# Chame a API Sheets
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId='1DTt9QS5MWHwXRHnCg3noxZD0-xpLo0zdQFGpNj_laNU',
                            range='Importações!A1:R').execute()
    values = result.get('values', [])
    df = pd.DataFrame(values)
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    df = df.drop(df.index[0])
    df = df.dropna(subset=['Empresa'])
except HttpError as err:
    print(err)

#Filtros
# df = df.loc[df['Empresa'] == 'LTEC']
df = df.loc[df['Empresa'] == 'LL4B']
df = df[df['NFTS SP'].isnull()]

df.insert(3, 'T_Registro', '4')
df.insert(4, 'T_Documento', '01')
df.insert(5, 'Serie', 'INV')
df.insert(6, 'SNFTS', 'N')
df.insert(7, 'Tributação', 'T')
df.insert(11, 'vDeducao', '000000000000000')
df.insert(4, 'Ind_CNPJ', '3')
df.insert(8, 'Regime_T', '0')
df.insert(11, 'Tipo_End', ' ')
df.insert(9, 'Complemento', ' ')
df.insert(12, 'CCM', '00000000')
df.insert(13, 'T_NFTS', '1')
df.insert(14, 'C_INSS', '0')
df.insert(15, 'N_OBRA', '0')
df.insert(15, 'C_RESERVADO', ' ')
df.insert(16, 'Data_PGTO', ' ')
df.insert(17, 'ISS Retido', '1')
df.insert(18, 'CNPJ', '0')
df.insert(18, 'LOGRADOURO', 'EUA')
df.insert(18, 'NÚMERO', ' ')
df.insert(18, 'BAIRRO', ' ')
df.insert(18, 'CIDADE', ' ')
df.insert(18, 'UF', ' ')
df.insert(18, 'CEP', '0')
df.insert(18, 'EMAIL', ' ')
df.insert(18, 'Descrição do Serviço', 'Prestação de serviços do exterior.')

df['% ISS'] = df['% ISS'].str.replace(',', '').str.replace('%', '')
df['Valor R$'] = df['Valor R$'].str.replace('.', '').str.replace(',', '').str.replace('R$', '')
df['Data Fechamento'] = df['Data Fechamento'].str.replace('/', '')
df['Cód. ISS'] = df['Cód. ISS'].str.replace('.', '')
df['Data Fechamento'] = pd.to_datetime(df['Data Fechamento'], format='%d%m%Y').dt.strftime('%Y %m %d')
df['Data Fechamento'] = df['Data Fechamento'].str.replace(' ', '')

#Parametrização das colunas utilizando 'PAD'
df['Invoice'] = df['Invoice'].str.pad(width=12, side='left', fillchar='0')
df['CNPJ'] = df['CNPJ'].str.pad(width=14, side='left', fillchar='0')
df['Valor R$'] = df['Valor R$'].str.pad(width=15, side='left', fillchar='0')
df['SUBITEM'] = df['SUBITEM'].str.pad(width=5, side='left', fillchar='0')
df['% ISS'] = df['% ISS'].apply(lambda x: str(x).zfill(4))
df['Fornecedor'] = df['Fornecedor'].astype(str).str.rjust(75)
df['LOGRADOURO'] = df['LOGRADOURO'].astype(str).str.rjust(50)
df['Serie'] = df['Serie'].astype(str).str.rjust(5)
df['NÚMERO'] = df['NÚMERO'].astype(str).str.rjust(12)
df['Complemento'] = df['Complemento'].astype(str).str.rjust(30)
df['BAIRRO'] = df['BAIRRO'].astype(str).str.rjust(30)
df['CIDADE'] = df['CIDADE'].astype(str).str.rjust(50)
df['EMAIL'] = df['EMAIL'].astype(str).str.rjust(75)
df['C_INSS'] = df['C_INSS'].str.pad(width=12, side='right', fillchar='0')
df['N_OBRA'] = df['N_OBRA'].str.pad(width=12, side='right', fillchar='0')
df['C_RESERVADO'] = df['C_RESERVADO'].astype(str).str.rjust(200)
df['Descrição do Serviço'] = df['Descrição do Serviço'].astype(str).str.rjust(200)
df['Data_PGTO'] = df['Data_PGTO'].str.pad(width=8, side='right', fillchar=' ')
df['UF'] = df['UF'].str.pad(width=2, side='right', fillchar=' ')
df['CEP'] = df['CEP'].str.pad(width=8, side='right', fillchar='0')

dfNFTS = pd.DataFrame()
dfNFTS[f'1002598710912023050120230531'] = pd.concat([df['T_Registro'], df['T_Documento'], df['Serie'], df['Invoice'], df['Data Fechamento'], df['SNFTS'],
df['Tributação'], df['Valor R$'], df['vDeducao'], df['SUBITEM'], df['Cód. ISS'], df['% ISS'],
df['ISS Retido'], df['Ind_CNPJ'], df['CNPJ'], df['CCM'], df['Fornecedor'], df['Tipo_End'], df['LOGRADOURO'],
df['NÚMERO'], df['Complemento'], df['BAIRRO'], df['CIDADE'], df['UF'], df['CEP'], df['EMAIL'],
df['T_NFTS'], df['Regime_T'], df['Data_PGTO'], df['C_INSS'], df['N_OBRA'], df['C_RESERVADO'], df['Descrição do Serviço']], 
axis=1, ignore_index=True).apply(lambda x: ''.join(str(val) for val in x), axis=1)

dfNFTS[f'1002598710912023050120230531'] = dfNFTS[f'1002598710912023050120230531'].str.pad(width=900, side='right', fillchar=' ')

dfNFTS.to_csv('LTECNFTS_IMP.txt', index=False)